## IMPORT THE MODULES

In [28]:
#import the necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import missingno as msno
from IPython.display import display
from sklearn.preprocessing import OrdinalEncoder
sns.set()
import warnings
warnings.filterwarnings('ignore')

## LOAD THE DATASET

In [29]:
#load the dataset
data = pd.read_csv('./DentalData311.csv')
data.sample(5)

,Timestamp,Patient Number,Chief complaint,Nature of Pain,Severity of pain,Onset and mode of pain,Factors which worsens the pain,Is the swelling painful?,Has the swelling changed since it was first noticed? If yes how quickly?,"Does the swelling changes during normal activities such as eating, speaking, etc?",...,Is there bleeding from the ulcer,Is there discharge from the ulcer?,Is there a foul smell from the ulcer?,Do the ulcers interfere with daily activities,Has the ulcer changed since first noticed?,Have you had similar ulcers?,Is there bleeding in the gums?,Is there pain in the gums,"If any tooth/teeth is/are mobile, what is the degree of mobility",Final Diagnosis
45,03/11/2023 11:24:33,M41,Pain,aching,Mild,The pain came suddenly,Eating,No,No,No,...,No,No,No,No,No,No,No,No,No,Dental caries with food impaction
43,03/11/2023 11:22:16,F35,Swelling,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,Chronic pulpits with swelling
48,03/11/2023 11:26:58,F31,Pain,aching,Moderate,lingering pain,Laying down,No,No,No,...,No,No,No,No,No,No,No,No,No,Acute pulpits
34,03/11/2023 11:02:46,M18,Pain,aching,Moderate,lingering pain,Hot foods or drinks,No,No,No,...,No,No,No,No,No,No,No,No,No,Acute pulpits
42,03/11/2023 11:21:13,F21,Pain,throbbing,Mild,The pain came suddenly,Cold foods or drinks,No,No,No,...,No,No,No,No,No,No,No,No,No,Reversible pulpits


## DROP FIELDS AND CLEAN UP

In [30]:
#Dropping unnecessary fields
data.drop('Timestamp',axis=1, inplace=True)
data.drop('Patient Number',axis=1, inplace=True)
data.sample(5)

,Chief complaint,Nature of Pain,Severity of pain,Onset and mode of pain,Factors which worsens the pain,Is the swelling painful?,Has the swelling changed since it was first noticed? If yes how quickly?,"Does the swelling changes during normal activities such as eating, speaking, etc?",Is the ulcer painful,Is there bleeding from the ulcer,Is there discharge from the ulcer?,Is there a foul smell from the ulcer?,Do the ulcers interfere with daily activities,Has the ulcer changed since first noticed?,Have you had similar ulcers?,Is there bleeding in the gums?,Is there pain in the gums,"If any tooth/teeth is/are mobile, what is the degree of mobility",Final Diagnosis
41,Swelling,aching,Mild,No,Laying down,No,Slow,No,No,No,No,No,No,No,No,No,No,No,Pulpits with swelling
40,Swelling,aching,Moderate,No,Laying down,No,Slow,No,No,No,No,No,No,No,No,No,No,No,Pulpits with swelling
30,Swelling,throbbing,Moderate,The pain came suddenly,Laying down,No,Slow,No,No,No,No,No,No,No,No,No,No,No,Pulpits with swelling
21,Pain,throbbing,Moderate,The pain came suddenly,Laying down,No,No,No,No,No,No,No,No,No,No,No,No,No,Acute pulpits
24,Pain,throbbing,Moderate,lingering pain,Laying down,No,Slow,No,No,No,No,No,No,No,No,No,No,No,Acute pulpits


## DATA PREPROCESSING

### USING ORDINAL ENCODER TO ENCODE THE FIELDS

In [31]:
#using ordinal encoder to change the fields to numerical values
encode=OrdinalEncoder(dtype=float)
patient_encode=pd.DataFrame(encode.fit_transform(data),columns=data.columns)
patient_encode.head(5)

,Chief complaint,Nature of Pain,Severity of pain,Onset and mode of pain,Factors which worsens the pain,Is the swelling painful?,Has the swelling changed since it was first noticed? If yes how quickly?,"Does the swelling changes during normal activities such as eating, speaking, etc?",Is the ulcer painful,Is there bleeding from the ulcer,Is there discharge from the ulcer?,Is there a foul smell from the ulcer?,Do the ulcers interfere with daily activities,Has the ulcer changed since first noticed?,Have you had similar ulcers?,Is there bleeding in the gums?,Is there pain in the gums,"If any tooth/teeth is/are mobile, what is the degree of mobility",Final Diagnosis
0,1.0,1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15.0
1,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0
2,2.0,4.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,16.0
3,1.0,4.0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
4,2.0,0.0,2.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14.0


In [32]:
patient_encode.to_csv('./DentalData311_encoded.csv',index=False)

In [33]:
train,test=train_test_split(patient_encode,test_size=0.3, random_state=42)
print(len(train))
print(len(test))

34
15


In [13]:
from sklearn.tree import DecisionTreeClassifier
m=DecisionTreeClassifier()

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Assume you have train, test splits 

X_train = train.drop(columns=['Final Diagnosis'])
print(X_train.shape)
y_train = train['Final Diagnosis']
print(y_train.shape)
print(X_train.isnull().sum())
print(y_train.isnull().sum())

X_test = test.drop(columns=['Final Diagnosis'])  
y_test = test['Final Diagnosis']

rf = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=42)
rf.estimators_ = [DecisionTreeClassifier() for _ in range(100)] 

rf.fit(X_train, y_train)

rf_pred = rf.predict(X_test)

rf_accuracy = rf.score(X_test, y_test) 

print("Random Forest Accuracy:",rf_accuracy)

(34, 18)
(34,)
Chief complaint                                                                      0
Nature of Pain                                                                       0
Severity of pain                                                                     0
Onset  and mode of pain                                                              0
Factors which worsens the pain                                                       0
Is the swelling painful?                                                             0
Has the swelling changed since it was first noticed? If yes how quickly?             0
Does the swelling changes during normal activities such as eating, speaking, etc?    0
Is the ulcer painful                                                                 0
Is there bleeding from the ulcer                                                     0
Is there discharge from the ulcer?                                                   0
Is there a foul smell from t

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').